# Importing the libraries

In [1]:
import pandas as pd
import numpy as np

# Importing the dataset

In [33]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [56]:
X = dataset.loc[:, dataset.columns != 'Exited'].drop(columns=['RowNumber','CustomerId','Surname'])
y = dataset.loc[:, ['Exited']].values

# Categorical data encoding

In [57]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [58]:
categorical_cols=X.dtypes == object
categorical_masked_cols = X.columns[categorical_cols].tolist()
categorical_masked_cols

['Geography', 'Gender']

In [59]:
ct = ColumnTransformer([('encoder', 
                         OneHotEncoder(categories='auto',drop='first'), 
                         categorical_masked_cols)], remainder='passthrough')
cat_data = np.array(ct.fit_transform(X[categorical_masked_cols]), dtype=np.int)
X = X.drop(columns=categorical_masked_cols).values
X = np.concatenate((X,cat_data), axis = 1)

# Splitting the dataset into the Training set and Test set

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [62]:
from sklearn.preprocessing import StandardScaler

In [63]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building ANN model

In [64]:
from keras.models import Sequential
from keras.layers import Dense

### Initialising the ANN

In [65]:
classifier = Sequential()

### Adding the input layer and the first hidden layer

In [66]:
classifier.add(Dense(units=int((X.shape[1] + y.shape[1]) /2) , 
                     input_dim=X.shape[1], 
                     kernel_initializer ='uniform', 
                     activation='relu'))

### Adding the second hidden layer

In [67]:
classifier.add(Dense(units=int((X.shape[1] + y.shape[1]) /2) , 
                     kernel_initializer ='uniform', 
                     activation='relu'))

### Adding the output layer

In [69]:
classifier.add(Dense(units=1 , 
                     kernel_initializer ='uniform', 
                     activation='sigmoid'))

### Compiling the ANN

In [70]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set

In [72]:
classifier.fit(X, y, epochs=10, batch_size=5)

Epoch 1/10
10000/10000 [==============================] - 10s 981us/step - loss: 0.5055 - accuracy: 0.7963
Epoch 2/10
10000/10000 [==============================] - 10s 971us/step - loss: 0.5057 - accuracy: 0.7963
Epoch 3/10
10000/10000 [==============================] - 10s 999us/step - loss: 0.5056 - accuracy: 0.7963
Epoch 4/10
10000/10000 [==============================] - 9s 870us/step - loss: 0.5056 - accuracy: 0.7963
Epoch 5/10
10000/10000 [==============================] - 8s 828us/step - loss: 0.5056 - accuracy: 0.7963
Epoch 6/10
10000/10000 [==============================] - 7s 657us/step - loss: 0.5056 - accuracy: 0.7963
Epoch 7/10
10000/10000 [==============================] - 7s 658us/step - loss: 0.5056 - accuracy: 0.7963
Epoch 8/10
10000/10000 [==============================] - 7s 663us/step - loss: 0.5056 - accuracy: 0.7963
Epoch 9/10
10000/10000 [==============================] - 7s 662us/step - loss: 0.5056 - accuracy: 0.7963
Epoch 10/10
10000/10000 [==================

# Predicting the Test set results

In [75]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Accuracy

In [76]:
from sklearn.metrics import confusion_matrix

In [77]:
cm = confusion_matrix(y_test, y_pred)

In [78]:
cm

array([[1595,    0],
       [ 405,    0]], dtype=int64)

In [81]:
1595/(1595+405) *100

79.75